# Notebook 1: Extraction 

The process conducted in this notebook involves reading the *candidates* dataset, transforming it into a Pandas DataFrame for easy manipulation, and then writing the processed data to a MySQL database. 

### Importing libraries and modules

The os and dotenv libraries are used to manage environment variables securely, allowing for the seamless loading of database credentials from a .env file. The sqlalchemy library, including its create_engine and text modules, provides a powerful ORM (Object-Relational Mapping) capability, enabling efficient interaction with the MySQL database. Finally, the pandas library is utilized to handle the candidates dataset, transforming it into a DataFrame for easy manipulation, analysis, and eventually writing it to the MySQL database.

In [1]:
import sys
import os
import numpy as np 
from dotenv import load_dotenv
from sqlalchemy import create_engine, text, types as sqltypes, Column
from sqlalchemy import Integer, SmallInteger, Float, Numeric, DateTime, Boolean, String, Time
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import declarative_base 
import pandas as pd

### Establishing the database connection

In [2]:
# Add the 'src' directory to the PYTHONPATH
sys.path.append(os.path.abspath('../src'))

# Importing the utility function from the db_utils module within the mypackage package
from connection.db_utils import get_db_connection, read_candidates_table

# Get the database connection
connection = get_db_connection()

Connected to the database successfully


### Reading the dataset and transforming it into a dataframe

In [ ]:
csv_path = "../data/candidates.csv"

df = pd.read_csv(csv_path, sep=";")


,First Name,Last Name,Email,Application Date,Country,YOE,Seniority,Technology,Code Challenge Score,Technical Interview Score
0,Bernadette,Langworth,leonard91@yahoo.com,26/02/2021,Norway,2,Intern,Data Engineer,3,3
1,Camryn,Reynolds,zelda56@hotmail.com,09/09/2021,Panama,10,Intern,Data Engineer,2,10
2,Larue,Spinka,okey_schultz41@gmail.com,14/04/2020,Belarus,4,Mid-Level,Client Success,10,9
3,Arch,Spinka,elvera_kulas@yahoo.com,01/10/2020,Eritrea,25,Trainee,QA Manual,7,1
4,Larue,Altenwerth,minnie.gislason@gmail.com,20/05/2020,Myanmar,13,Mid-Level,Social Media Community Management,9,7
...,...,...,...,...,...,...,...,...,...,...
49995,Bethany,Shields,rocky_mitchell@hotmail.com,09/01/2022,Dominican Republic,27,Trainee,Security,2,1
49996,Era,Swaniawski,dolores.roob@hotmail.com,02/06/2020,Morocco,21,Lead,Game Development,1,2
49997,Martin,Lakin,savanah.stracke@gmail.com,15/12/2018,Uganda,20,Trainee,System Administration,6,1
49998,Aliya,Abernathy,vivienne.fritsch@yahoo.com,30/05/2020,Czech Republic,20,Senior,Database Administration,0,0


### Data profiling

#### Profiling data types

In [4]:
def pandas_to_mysql_type(pandas_dtype, sample_data=None):
    """Convert Pandas dtype to the most appropriate MySQL type with specific lengths."""
    
    if pd.api.types.is_integer_dtype(pandas_dtype):
        # Determine best integer type based on sample values
        if sample_data is not None:
            min_val, max_val = int(sample_data.min()), int(sample_data.max())
            if 0 <= max_val <= 255:
                return SmallInteger  # Equivalent to TINYINT
            elif -32768 <= min_val <= 32767:
                return SmallInteger  # Equivalent to SMALLINT
            elif -2147483648 <= min_val <= 2147483647:
                return Integer  # Equivalent to INT
            else:
                return Integer  # Equivalent to BIGINT
        return Integer  # Default if no sample data is provided

    elif pd.api.types.is_float_dtype(pandas_dtype):
        return Numeric(10, 2)  # DECIMAL(10,2) for better precision

    elif pd.api.types.is_datetime64_any_dtype(pandas_dtype):
        return DateTime

    elif pd.api.types.is_timedelta64_dtype(pandas_dtype):
        return Time

    elif pd.api.types.is_bool_dtype(pandas_dtype):
        return Boolean

    elif isinstance(pandas_dtype, pd.CategoricalDtype) and sample_data is not None:
        # Estimate max length for categorical variables
        max_length = max(sample_data.astype(str).apply(len), default=255)
        return String(max_length)

    elif pd.api.types.is_object_dtype(pandas_dtype) and sample_data is not None:
        # Estimate max length for object (string) columns
        max_length = max(sample_data.astype(str).apply(len), default=255)
        return String(max_length)

    return String(255)  # Default fallback for unknown types


for col_name in df.columns:
    dtype = df[col_name].dtype
    mysql_type = pandas_to_mysql_type(dtype, df[col_name])
    print(f"Column '{col_name}': Pandas dtype = {dtype}, Suggested MySQL type = {mysql_type}")

Column 'First Name': Pandas dtype = object, Suggested MySQL type = VARCHAR(11)
Column 'Last Name': Pandas dtype = object, Suggested MySQL type = VARCHAR(13)
Column 'Email': Pandas dtype = object, Suggested MySQL type = VARCHAR(36)
Column 'Application Date': Pandas dtype = object, Suggested MySQL type = VARCHAR(10)
Column 'Country': Pandas dtype = object, Suggested MySQL type = VARCHAR(51)
Column 'YOE': Pandas dtype = int64, Suggested MySQL type = <class 'sqlalchemy.sql.sqltypes.SmallInteger'>
Column 'Seniority': Pandas dtype = object, Suggested MySQL type = VARCHAR(9)
Column 'Technology': Pandas dtype = object, Suggested MySQL type = VARCHAR(39)
Column 'Code Challenge Score': Pandas dtype = int64, Suggested MySQL type = <class 'sqlalchemy.sql.sqltypes.SmallInteger'>
Column 'Technical Interview Score': Pandas dtype = int64, Suggested MySQL type = <class 'sqlalchemy.sql.sqltypes.SmallInteger'>


#### Profiling the lenght of numerical values

In [5]:
# Display the maximum values of numeric columns
print("Maximum values in numeric columns:")
print(df.max(numeric_only=True))

Maximum values in numeric columns:
YOE                          30
Code Challenge Score         10
Technical Interview Score    10
dtype: int64


#### Profilingh the lenght of the string values

In [6]:
text_columns = df.select_dtypes(include=['object'])

# Calculate the length of each string in the text columns
lengths = text_columns.apply(lambda col: col.map(lambda x: len(str(x))))

# Find the maximum string lengths for each text column
max_lengths = lengths.max()

# Display the results
print("Maximum string lengths in text columns:")
print(max_lengths)


Maximum string lengths in text columns:
First Name          11
Last Name           13
Email               36
Application Date    10
Country             51
Seniority            9
Technology          39
dtype: int64


### "candidates" table creation

In [17]:
def clean_column_names(df):
    """Cleans DataFrame column names to match database conventions."""
    cleaned_columns = []
    for col in df.columns:
        cleaned_col = col.replace(" ", "_").lower()  # Replace spaces with underscores, lowercase
        cleaned_columns.append(cleaned_col)
    df.columns = cleaned_columns
    return df

clean_column_names(df)

df.head(3)

,first_name,last_name,email,application_date,country,yoe,seniority,technology,code_challenge_score,technical_interview_score
0,Bernadette,Langworth,leonard91@yahoo.com,26/02/2021,Norway,2,Intern,Data Engineer,3,3
1,Camryn,Reynolds,zelda56@hotmail.com,09/09/2021,Panama,10,Intern,Data Engineer,2,10
2,Larue,Spinka,okey_schultz41@gmail.com,14/04/2020,Belarus,4,Mid-Level,Client Success,10,9


In [18]:
def get_engine():
    """SQLAlchemy engine for MySQL connection."""
    return create_engine(
        f"mysql+mysqlconnector://{os.getenv('MYSQL_USER')}:{os.getenv('MYSQL_PASSWORD')}@{os.getenv('MYSQL_HOST')}:{os.getenv('MYSQL_PORT')}/{os.getenv('MYSQL_DB')}"
    )

In [19]:
# Base class for ORM models
Base = declarative_base()

# Define the table structure
class Applicants(Base):
    __tablename__ = 'candidates'  # Table name

    id = Column(Integer, primary_key=True, autoincrement=True)
    first_name = Column(String(20), nullable=False)  # 
    last_name = Column(String(20), nullable=False)   
    email = Column(String(50), nullable=False)         
    application_date = Column(String(20), nullable=False) 
    country = Column(String(60), nullable=False)      
    yoe = Column(SmallInteger, nullable=False)           
    seniority = Column(String(15), nullable=False)   
    technology = Column(String(50), nullable=False)   
    code_challenge_score = Column(SmallInteger, nullable=False) 
    technical_interview_score = Column(SmallInteger, nullable=False) 

# Function to create the table
def create_tables():
    engine = get_engine()
    Base.metadata.create_all(engine)
    print("Table 'candidates' created successfully!")

if __name__ == "__main__":
    create_tables()


Table 'candidates' created successfully!


### Data migration to MySQL "ws_001" database

In [20]:
def insert_data(df, table_name, batch_size=1000):
    """
    Inserts data from DataFrame to DB in MySQL in batches.

    :param df: DataFrame with data to insert.
    :param table_name: Name of the table where data is supposed to be inserted.
    :param batch_size: Number of rows to insert per batch.
    """
    engine = get_engine()
    connection = engine.connect()
    transaction = connection.begin()
    print("Successful connection.")

    try:
        for i in range(0, len(df), batch_size):
            batch_df = df.iloc[i:i + batch_size]
            batch_df.to_sql(name=table_name, con=connection, if_exists='append', index=False)
            print(f"Inserted batch {i // batch_size + 1} successfully.")

        transaction.commit()
        print(f"All data inserted into '{table_name}' successfully.")
    
    except Exception as e:
        transaction.rollback()
        print(f"Error: {e}")
    
    finally:
        connection.close()

insert_data(df, "candidates")



Successful connection.
Inserted batch 1 successfully.
Inserted batch 2 successfully.
Inserted batch 3 successfully.
Inserted batch 4 successfully.
Inserted batch 5 successfully.
Inserted batch 6 successfully.
Inserted batch 7 successfully.
Inserted batch 8 successfully.
Inserted batch 9 successfully.
Inserted batch 10 successfully.
Inserted batch 11 successfully.
Inserted batch 12 successfully.
Inserted batch 13 successfully.
Inserted batch 14 successfully.
Inserted batch 15 successfully.
Inserted batch 16 successfully.
Inserted batch 17 successfully.
Inserted batch 18 successfully.
Inserted batch 19 successfully.
Inserted batch 20 successfully.
Inserted batch 21 successfully.
Inserted batch 22 successfully.
Inserted batch 23 successfully.
Inserted batch 24 successfully.
Inserted batch 25 successfully.
Inserted batch 26 successfully.
Inserted batch 27 successfully.
Inserted batch 28 successfully.
Inserted batch 29 successfully.
Inserted batch 30 successfully.
Inserted batch 31 successf